In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [12]:
#parameters
learning_rate = 0.0001
training_epochs = 5
batch_size = 8

In [24]:
trans = transforms.Compose([transforms.ToTensor()])

train_data = dsets.ImageFolder(root='./dogscats/train_data', transform=trans)

In [5]:
#Dataloader
data_loader = DataLoader(dataset = train_data, batch_size = batch_size,
                         shuffle = True, num_workers=2, drop_last=True)

In [6]:
#model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(),
            nn.Linear(120, 2),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        
        return out


In [7]:
#testing 
model = CNN().to(device)
test_input = (torch.Tensor(3,3,28,28)).to(device)
test_out = model(test_input)

In [8]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=2, bias=True)
  )
)

In [9]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [10]:
#optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for num, data in enumerate(data_loader): 
        imgs, labels = data
        imgs = imgs.to(device) #input data
        labels = labels.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(imgs) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, labels) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.6931071877479553
[Epoch:2] cost = 0.6930043697357178
[Epoch:3] cost = 0.6928980350494385
[Epoch:4] cost = 0.6928015947341919
[Epoch:5] cost = 0.6927076578140259
Learning Finished!


In [17]:
trans=torchvision.transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./dogscats/test_data', transform=trans)

In [18]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [20]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = model(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.512499988079071
